<a href="https://colab.research.google.com/github/huuvu18021438/Human_Reid/blob/main/Human_Reid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 9809 (delta 188), reused 267 (delta 183), pack-reused 9526
Receiving objects: 100% (9809/9809), 9.59 MiB | 23.72 MiB/s, done.
Resolving deltas: 100% (7226/7226), done.


In [ ]:
%cd deep-person-reid/
!pip install -r requirements.txt

/content/deep-person-reid
     |████████████████████████████████| 5.8 MB 8.9 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 185 kB 45.7 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 42 kB 870 kB/s 
     |████████████████████████████████| 69 kB 7.9 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 41 kB 615 kB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 


In [ ]:
!python setup.py develop

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
writing manifest file 'torchreid.egg-info/SOURCES.txt'

In [ ]:
import torchreid

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3508 KB/s, 42 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
model = torchreid.models.build_model(
      name='osnet_x1_0',
      num_classes=datamanager.num_train_pids,
      loss='softmax',
      pretrained=True
  )
  
model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
      model,
      optim='adam',
      lr=0.0003
  )

scheduler = torchreid.optim.build_lr_scheduler(
      optimizer,
      lr_scheduler='single_step',
      stepsize=20
  )

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
  )

engine.run(
    save_dir='log/osnet_x1_0',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False
  )

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 61.4MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.486 (1.062)	data 0.001 (0.557)	eta 2:22:48	loss 6.6484 (6.6329)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/20][20/404]	time 0.489 (0.776)	data 0.001 (0.279)	eta 1:44:12	loss 6.6332 (6.6342)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/20][30/404]	time 0.488 (0.681)	data 0.001 (0.186)	eta 1:31:22	loss 6.6216 (6.6263)	acc 0.0000 (0.5208)	lr 0.000300
epoch: [1/20][40/404]	time 0.488 (0.633)	data 0.000 (0.140)	eta 1:24:51	loss 6.5863 (6.6209)	acc 3.1250 (0.8594)	lr 0.000300
epoch: [1/20][50/404]	time 0.496 (0.605)	data 0.000 (0.112)	eta 1:20:57	loss 6.5295 (6.5919)	acc 3.1250 (1.0000)	lr 0.000300
epoch: [1/20][60/404]	time 0.489 (0.587)	data 0.000 (0.094)	eta 1:18:23	loss 6.5218 (6.5689)	acc 0.0000 (1.2500)	lr 0.000300
epoch: [1/20][70/404]	time 0.489 (0.573)	data 0.000 (0.080)	eta 1:16:29	loss 6.2335 (6.5371)	acc 3.1250 (1.6964)	lr 0.000300
epoch: [1/20][80/404]	time 0.495 (0.563)	data 0.000 (0.070)	eta 1:15:05	loss 5.9866 (6.5004)	acc 3.1250 (2.1094)	lr 0.000300


In [ ]:
#kết quả truy vấn và tệp hình ảnh dự đoán
engine.run(
    save_dir='visrank_test_result',
    test_only=True,
    visrank=True
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0678 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 71.6%
CMC curve
Rank-1  : 89.1%
Rank-5  : 95.7%
Rank-10 : 97.3%
Rank-20 : 98.3%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "visrank_test_result/visrank_market1501" ...


In [ ]:
#Heat map trích xuất các tính năng cho Reid
!python tools/visualize_actmap.py \
--root $DATA/reid \
-d market1501 \
-m osnet_x1_0 \
--weights /content/deep-person-reid/log/osnet_x1_0/model/model.pth.tar-20 \
--save-dir log/visactmap_osnet_x1_0_market1501

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/reid/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/reid/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 12635 KB/s, 11 seconds passed
Extracting "/reid/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/to

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
!tensorboard dev upload \
  --logdir log/osnet_x1_0 \
  --name "My result" \
  --description "Human Re_Identification" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

log/osnet_x1_0

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=jp7lYny64KRkm3GvKclKpT3rlhCO8C&prompt=consent&access_type=offlin